In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import pathway as pw
from datetime import datetime
import panel as pn
import bokeh.plotting

In [5]:
df = pd.read_csv('dataset.csv')
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)

df[["Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType", "Latitude", "Longitude", "SystemCodeNumber"]].to_csv("parking_stream.csv", index=False)

In [6]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str
    Latitude: float
    Longitude: float
    SystemCodeNumber: str

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [7]:
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [8]:
def baseline_pricing(price, occupancy, capacity, alpha=0.1):
    return price + alpha * (occupancy / capacity)


In [9]:
baseline_window = data_with_time.with_columns(
    t = pw.this.t,
    model = "Baseline",
    price = baseline_pricing(
        price=10,
        occupancy=pw.this.Occupancy,
        capacity=pw.this.Capacity
    )
)


In [10]:
def demand_based_pricing(base_price, occupancy, capacity, queue_length, traffic, is_special_day, vehicle_type,
                          alpha=1.0, beta=0.5, gamma=0.3, delta=0.7, epsilon=0.2, lambda_=0.4):

    traffic_val = pw.if_else(traffic == 'low', 1,
                   pw.if_else(traffic == 'medium', 2,
                   pw.if_else(traffic == 'high', 3, 2)))

    vehicle_val = pw.if_else(vehicle_type == 'bike', 0.5,
                    pw.if_else(vehicle_type == 'car', 1.0,
                    pw.if_else(vehicle_type == 'truck', 1.5, 1.0)))


    demand = (alpha * (occupancy / capacity) +
              beta * queue_length -
              gamma * traffic_val +
              delta * is_special_day +
              epsilon * vehicle_val)


    normalized_demand = pw.if_else(demand / 10.0 < 0, 0,
                          pw.if_else(demand / 10.0 > 1, 1, demand / 10.0))

    price = base_price * (1 + lambda_ * normalized_demand)

    final_price = pw.if_else(price < base_price * 0.5, base_price * 0.5,
                    pw.if_else(price > base_price * 2, base_price * 2, price))

    return final_price

In [11]:
import datetime
data_with_price = data_with_time.with_columns(
    price = demand_based_pricing(
        base_price=10,
        occupancy=pw.this.Occupancy,
        capacity=pw.this.Capacity,
        queue_length=pw.this.QueueLength,
        traffic=pw.this.TrafficConditionNearby,
        is_special_day=pw.this.IsSpecialDay,
        vehicle_type=pw.this.VehicleType
    )
)

delta_window = (
    data_with_price.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        sum_price=pw.reducers.sum(pw.this.price),
        count_price=pw.reducers.count(pw.this.price)
    )
    .with_columns(
        avg_price=pw.this.sum_price / pw.this.count_price
    )
)

In [14]:
pn.extension()
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400, width=800, title="Baseline Pricing",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="blue")
    fig.circle("t", "price", source=source, size=5, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [13]:
pn.extension()
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400, width=800, title="Dynamic Pricing",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="blue")
    fig.circle("t", "price", source=source, size=5, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)